# RAG Evaluation

## Code to generate response from previous response

In [ ]:
from openai import OpenAI
import json
client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

json_file_path = "courses.json"
with open(json_file_path, "r") as f:
    course_data = str(json.load(f))

# course_data = json_content.dumps()

prompt_init = f'''Hello You are a course advisor. Accoring to the prompt use the following information
                to reccomend courses to the student : {course_data}. Feel free to ask more questions to get more information 
                and provide better recommendations. Provide a summary of the course description, topics and 
                technologies taught and the pre requisites required to take the course.'''

In [21]:
from openai import OpenAI
import json

# Initialize the OpenAI client with your API key
client = OpenAI()
model = "gpt-3.5-turbo"
course_data = load_course_data("courses.json")
# Load course data from a JSON file
def load_course_data(json_file_path):
    with open(json_file_path, "r") as f:
        return json.load(f)

# Function to generate course recommendations
def get_course_recommendations(prompt):
    # Load the course data

    
    # Prepare the initial system message
    prompt_init = f'''Hello You are a course advisor. According to the prompt use the following information
                    to recommend courses to the student: {course_data}. Feel free to ask more questions to get more information 
                    and provide better recommendations. Provide a summary of the course description, topics and 
                    technologies taught and the prerequisites required to take the course.
                    '''
    
    # Prepare the messages
    messages = [
        {"role": "system", "content": prompt_init},
        {"role": "user", "content": prompt}
    ]
    
    # Send the messages to the OpenAI API and get the response
    stream = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            stream=False,
        )
    response = stream
    return response.choices[0].message.content



# Generate Test Prompt

In [13]:
import pandas as pd

data = [
    {"Prompt ID": 1, "Prompt": "Can you recommend a course that covers the basics of web development?"},
    {"Prompt ID": 2, "Prompt": "I'm interested in learning Python for application engineering. Can you suggest a course?"},
    {"Prompt ID": 3, "Prompt": "What course should I take if I want to understand data science engineering methods and tools?"},
    {"Prompt ID": 4, "Prompt": "Do you have any courses that focus on business analysis and information engineering?"},
    {"Prompt ID": 5, "Prompt": "Can you suggest a course that teaches neural modeling methods and tools?"},
    {"Prompt ID": 6, "Prompt": "I'm a beginner with no programming experience. What courses should I start with?"},
    {"Prompt ID": 7, "Prompt": "Are there any courses that cover machine learning algorithms and their implementation?"},
    {"Prompt ID": 8, "Prompt": "Can you recommend a course on engineering high-performance coding for fintech?"},
    {"Prompt ID": 9, "Prompt": "Do you offer courses that provide an introduction to the fundamental techniques of web design and user experience?"},
    {"Prompt ID": 10, "Prompt": "What course should I take if I want to learn about advanced big-data applications and indexing techniques?"},
]

ideal_responses = [
    "INFO 6150. Web Design and User Experience Engineering. (4 Hours). This course covers both conceptual and technical aspects of Web design, including user experience design, web standards, best practices, information architecture, usability, and more.",
    "INFO 5002. Introduction to Python for Information Systems. (4 Hours). This course studies the Python programming language for application engineering, covering core concepts and skills for building applications using Python.",
    "INFO 6105. Data Science Engineering Methods and Tools. (4 Hours). This course introduces fundamental techniques for machine learning and data science engineering, including supervised and unsupervised learning, neural networks, and more.",
    "INFO 6215. Business Analysis and Information Engineering. (4 Hours). This course covers computer information systems, system development life cycle, system modeling and analysis, and effective leverage of UML for software solutions.",
    "INFO 6106. Neural Modeling Methods and Tools. (4 Hours). This course uses a graph theoretic approach to build models representing dependencies of model components, explaining neural models and how to interact with machines using them.",
    "INFO 5002. Introduction to Python for Information Systems. (4 Hours). This hands-on course offers proficiency in Python core concepts and skills for building applications using Python.",
    "INFO 6105. Data Science Engineering Methods and Tools. (4 Hours). This course introduces fundamental techniques for machine learning and data science engineering, including supervised and unsupervised learning, neural networks, and more.",
    "INFO 7110. High-Performance Coding for Fintech. (4 Hours). This course covers high-performance computing challenges and solutions in the fintech space using Java, C++, MATLAB, R, and Python.",
    "INFO 6150. Web Design and User Experience Engineering. (4 Hours). This course covers both conceptual and technical aspects of Web design, including user experience design, web standards, best practices, information architecture, usability, and more.",
    "INFO 7255. Advanced Big-Data Applications and Indexing Techniques. (4 Hours). This course studies advanced indexing techniques and algorithms for big-data platforms such as Hadoop and NoSQL databases."
]

# Generate the responses and append to data
for i, item in enumerate(data):
    generated_response = get_course_recommendations(item["Prompt"])
    item["Ideal Response"] = ideal_responses[i]
    item["Generated Response"] = generated_response

# Create a DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("course_recommendation_evaluation.csv", index=False)

print("CSV file 'course_recommendation_evaluation.csv' has been created.")

df = pd.DataFrame(data)
df.to_csv("course_recommendation_prompts.csv", index=False)

print("CSV file 'course_recommendation_prompts.csv' has been created.")

CSV file 'course_recommendation_evaluation.csv' has been created.
CSV file 'course_recommendation_prompts.csv' has been created.


In [24]:
from openai import OpenAI
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer

# Initialize the OpenAI client with your API key
client = OpenAI()

# Load course data from a JSON file
def load_course_data(json_file_path):
    with open(json_file_path, "r") as f:
        return json.load(f)

# Function to generate course recommendations
# def get_course_recommendations(prompt):
#     # Load the course data
#     course_data = load_course_data("courses.json")
    
#     # Prepare the initial system message
#     prompt_init = f'''Hello You are a course advisor. According to the prompt use the following information
#                     to recommend courses to the student: {course_data}. Feel free to ask more questions to get more information 
#                     and provide better recommendations. Provide a summary of the course description, topics and 
#                     technologies taught and the prerequisites required to take the course.
#                     '''
    
#     # Prepare the messages
#     messages = [
#         {"role": "system", "content": prompt_init},
#         {"role": "user", "content": prompt}
#     ]
    
#     # Send the messages to the OpenAI API and get the response
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=messages
#     )
    
#     # Extract and return the assistant's reply
#     reply = response['choices'][0]['message']['content']
#     return reply


# Calculate Exact Match
def calculate_exact_match(ideal_response, generated_response):
    return 1 if ideal_response.strip().lower() == generated_response.strip().lower() else 0

# Calculate BLEU Score
def calculate_bleu_score(ideal_response, generated_response):
    reference = [ideal_response.split()]
    candidate = generated_response.split()
    return sentence_bleu(reference, candidate)

# Calculate ROUGE Scores
def calculate_rouge_scores(ideal_response, generated_response):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ideal_response, generated_response)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

# Calculate Cosine Similarity
def calculate_cosine_similarity(ideal_response, generated_response):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([ideal_response, generated_response])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# Initialize the data
data = [
    {"Prompt ID": 1, "Prompt": "Can you recommend a course that covers the basics of web development?"},
    {"Prompt ID": 2, "Prompt": "I'm interested in learning Python for application engineering. Can you suggest a course?"},
    {"Prompt ID": 3, "Prompt": "What course should I take if I want to understand data science engineering methods and tools?"},
    {"Prompt ID": 4, "Prompt": "Do you have any courses that focus on business analysis and information engineering?"},
    {"Prompt ID": 5, "Prompt": "Can you suggest a course that teaches neural modeling methods and tools?"},
    {"Prompt ID": 6, "Prompt": "I'm a beginner with no programming experience. What courses should I start with?"},
    {"Prompt ID": 7, "Prompt": "Are there any courses that cover machine learning algorithms and their implementation?"},
    {"Prompt ID": 8, "Prompt": "Can you recommend a course on engineering high-performance coding for fintech?"},
    {"Prompt ID": 9, "Prompt": "Do you offer courses that provide an introduction to the fundamental techniques of web design and user experience?"},
    {"Prompt ID": 10, "Prompt": "What course should I take if I want to learn about advanced big-data applications and indexing techniques?"},
]

ideal_responses = [
    "INFO 6150. Web Design and User Experience Engineering. (4 Hours). This course covers both conceptual and technical aspects of Web design, including user experience design, web standards, best practices, information architecture, usability, and more.",
    "INFO 5002. Introduction to Python for Information Systems. (4 Hours). This course studies the Python programming language for application engineering, covering core concepts and skills for building applications using Python.",
    "INFO 6105. Data Science Engineering Methods and Tools. (4 Hours). This course introduces fundamental techniques for machine learning and data science engineering, including supervised and unsupervised learning, neural networks, and more.",
    "INFO 6215. Business Analysis and Information Engineering. (4 Hours). This course covers computer information systems, system development life cycle, system modeling and analysis, and effective leverage of UML for software solutions.",
    "INFO 6106. Neural Modeling Methods and Tools. (4 Hours). This course uses a graph theoretic approach to build models representing dependencies of model components, explaining neural models and how to interact with machines using them.",
    "INFO 5002. Introduction to Python for Information Systems. (4 Hours). This hands-on course offers proficiency in Python core concepts and skills for building applications using Python.",
    "INFO 6105. Data Science Engineering Methods and Tools. (4 Hours). This course introduces fundamental techniques for machine learning and data science engineering, including supervised and unsupervised learning, neural networks, and more.",
    "INFO 7110. High-Performance Coding for Fintech. (4 Hours). This course covers high-performance computing challenges and solutions in the fintech space using Java, C++, MATLAB, R, and Python.",
    "INFO 6150. Web Design and User Experience Engineering. (4 Hours). This course covers both conceptual and technical aspects of Web design, including user experience design, web standards, best practices, information architecture, usability, and more.",
    "INFO 7255. Advanced Big-Data Applications and Indexing Techniques. (4 Hours). This course studies advanced indexing techniques and algorithms for big-data platforms such as Hadoop and NoSQL databases."
]

# Generate the responses and append to data
for i, item in enumerate(data):
    generated_response = get_course_recommendations(item["Prompt"])
    ideal_response = ideal_responses[i]
    
    item["Ideal Response"] = ideal_response
    item["Generated Response"] = generated_response
    
    # Calculate metrics
    item["Exact Match"] = calculate_exact_match(ideal_response, generated_response)
    item["BLEU Score"] = calculate_bleu_score(ideal_response, generated_response)
    rouge1, rouge2, rougel = calculate_rouge_scores(ideal_response, generated_response)
    item["ROUGE-1"] = rouge1
    item["ROUGE-2"] = rouge2
    item["ROUGE-L"] = rougel
    item["Cosine Similarity"] = calculate_cosine_similarity(ideal_response, generated_response)

# Create a DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("course_recommendation_evaluation.csv", index=False)

print("CSV file 'course_recommendation_evaluation.csv' has been created.")

# The following line was duplicating the creation of a similar CSV. It's unnecessary, so I'm commenting it out.
# df = pd.DataFrame(data)
# df.to_csv("course_recommendation_prompts.csv", index=False)

# You can print the DataFrame to check the results
print(df)


/home/snehilaryan/rag_eval_back/rag_evaluation_shopping_chatbot/myenv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [25]:

df

,Prompt ID,Prompt,Ideal Response,Generated Response
0,1,Can you recommend a course that covers the bas...,INFO 6150. Web Design and User Experience Engi...,"I recommend taking the course ""INFO 502. Intro..."
1,2,I'm interested in learning Python for applicat...,INFO 5002. Introduction to Python for Informat...,Based on your interest in learning Python for ...
2,3,What course should I take if I want to underst...,INFO 6105. Data Science Engineering Methods an...,You should consider taking the course INFO 610...
3,4,Do you have any courses that focus on business...,INFO 6215. Business Analysis and Information E...,"Yes, we have a course that focuses on business..."
4,5,Can you suggest a course that teaches neural m...,INFO 6106. Neural Modeling Methods and Tools. ...,Sure! I recommend the course INFO 6106 - Neura...
5,6,I'm a beginner with no programming experience....,INFO 5002. Introduction to Python for Informat...,"As a beginner with no programming experience, ..."
6,7,Are there any courses that cover machine learn...,INFO 6105. Data Science Engineering Methods an...,"Yes, there is a course that covers machine lea..."
7,8,Can you recommend a course on engineering high...,INFO 7110. High-Performance Coding for Fintech...,I recommend taking the course INFO 07110 - Hig...
8,9,Do you offer courses that provide an introduct...,INFO 6150. Web Design and User Experience Engi...,"Yes, we offer a course called ""INFO 5100. Appl..."
9,10,What course should I take if I want to learn a...,INFO 7255. Advanced Big-Data Applications and ...,The course you should take to learn about adva...


In [26]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer

# Assuming the existing DataFrame 'df' is already created with the necessary columns
# df = pd.read_csv("course_recommendation_evaluation.csv") # Uncomment if you need to load the df from a file

# Function to calculate Exact Match
def calculate_exact_match(ideal_response, generated_response):
    return 1 if ideal_response.strip().lower() == generated_response.strip().lower() else 0

# Function to calculate BLEU Score
def calculate_bleu_score(ideal_response, generated_response):
    reference = [ideal_response.split()]
    candidate = generated_response.split()
    return sentence_bleu(reference, candidate)

# Function to calculate ROUGE Scores
def calculate_rouge_scores(ideal_response, generated_response):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ideal_response, generated_response)
    return scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure

# Function to calculate Cosine Similarity
def calculate_cosine_similarity(ideal_response, generated_response):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([ideal_response, generated_response])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# Calculate metrics for each row and add to the DataFrame
df["Exact Match"] = df.apply(lambda row: calculate_exact_match(row["Ideal Response"], row["Generated Response"]), axis=1)
df["BLEU Score"] = df.apply(lambda row: calculate_bleu_score(row["Ideal Response"], row["Generated Response"]), axis=1)
df["ROUGE-1"], df["ROUGE-2"], df["ROUGE-L"] = zip(*df.apply(lambda row: calculate_rouge_scores(row["Ideal Response"], row["Generated Response"]), axis=1))
df["Cosine Similarity"] = df.apply(lambda row: calculate_cosine_similarity(row["Ideal Response"], row["Generated Response"]), axis=1)

# Save the modified DataFrame to a new CSV file
df.to_csv("course_recommendation_evaluation_with_metrics.csv", index=False)

# Print the updated DataFrame to check the results
print(df)


   Prompt ID                                             Prompt  \
0          1  Can you recommend a course that covers the bas...   
1          2  I'm interested in learning Python for applicat...   
2          3  What course should I take if I want to underst...   
3          4  Do you have any courses that focus on business...   
4          5  Can you suggest a course that teaches neural m...   
5          6  I'm a beginner with no programming experience....   
6          7  Are there any courses that cover machine learn...   
7          8  Can you recommend a course on engineering high...   
8          9  Do you offer courses that provide an introduct...   
9         10  What course should I take if I want to learn a...   

                                      Ideal Response  \
0  INFO 6150. Web Design and User Experience Engi...   
1  INFO 5002. Introduction to Python for Informat...   
2  INFO 6105. Data Science Engineering Methods an...   
3  INFO 6215. Business Analysis and In

/home/snehilaryan/rag_eval_back/rag_evaluation_shopping_chatbot/myenv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/snehilaryan/rag_eval_back/rag_evaluation_shopping_chatbot/myenv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [27]:
df

,Prompt ID,Prompt,Ideal Response,Generated Response,Exact Match,BLEU Score,ROUGE-1,ROUGE-2,ROUGE-L,Cosine Similarity
0,1,Can you recommend a course that covers the bas...,INFO 6150. Web Design and User Experience Engi...,"I recommend taking the course ""INFO 502. Intro...",0,2.435896e-155,0.194444,0.028169,0.111111,0.159952
1,2,I'm interested in learning Python for applicat...,INFO 5002. Introduction to Python for Informat...,Based on your interest in learning Python for ...,0,1.551752e-01,0.386667,0.337838,0.386667,0.658137
2,3,What course should I take if I want to underst...,INFO 6105. Data Science Engineering Methods an...,You should consider taking the course INFO 610...,0,1.160465e-01,0.341772,0.294872,0.341772,0.646264
3,4,Do you have any courses that focus on business...,INFO 6215. Business Analysis and Information E...,"Yes, we have a course that focuses on business...",0,6.068108e-02,0.278607,0.201005,0.258706,0.598856
4,5,Can you suggest a course that teaches neural m...,INFO 6106. Neural Modeling Methods and Tools. ...,Sure! I recommend the course INFO 6106 - Neura...,0,1.661102e-01,0.407895,0.346667,0.394737,0.663625
5,6,I'm a beginner with no programming experience....,INFO 5002. Introduction to Python for Informat...,"As a beginner with no programming experience, ...",0,1.737432e-02,0.186916,0.084906,0.130841,0.268323
6,7,Are there any courses that cover machine learn...,INFO 6105. Data Science Engineering Methods an...,"Yes, there is a course that covers machine lea...",0,7.565287e-02,0.293194,0.222222,0.282723,0.597665
7,8,Can you recommend a course on engineering high...,INFO 7110. High-Performance Coding for Fintech...,I recommend taking the course INFO 07110 - Hig...,0,8.189180e-02,0.304094,0.213018,0.269006,0.559109
8,9,Do you offer courses that provide an introduct...,INFO 6150. Web Design and User Experience Engi...,"Yes, we offer a course called ""INFO 5100. Appl...",0,7.512136e-79,0.224719,0.113636,0.191011,0.429951
9,10,What course should I take if I want to learn a...,INFO 7255. Advanced Big-Data Applications and ...,The course you should take to learn about adva...,0,1.218564e-01,0.290155,0.251309,0.290155,0.662588
